In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd

from utils.Block import Model
from utils.train import train

In [2]:
model = Model([64, 64], 13)

In [3]:
from sklearn.utils import class_weight

In [ ]:
import pathlib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
import tensorflow as tf

from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import make_scorer
from sklearn import ensemble, linear_model
from xgboost.sklearn import XGBClassifier

from catboost import Pool, CatBoostClassifier

X_submit = pd.read_csv('./data/420_X_submit.csv', index_col='id')
X_train = pd.read_csv('./data/420_X_train.csv', index_col='id')
X_test = pd.read_csv('./data/420_X_test.csv', index_col='id')

y_submit = pd.read_csv('./data/004_test.csv', index_col='id')
y_train = pd.read_csv('./data/410_y_train.csv', index_col='id')
y_test = pd.read_csv('./data/410_y_test.csv', index_col='id')

In [5]:
y_ = pd.read_csv('./data/003_train.csv', index_col='id')

In [6]:
class_w = class_weight.compute_class_weight('balanced', np.unique(y_), y_.values.reshape(-1))

In [7]:
np.unique(y_)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12])

In [8]:
class_w

array([1.11321385e+01, 3.53182171e+00, 3.47722073e-01, 2.81728234e-01,
       4.32613896e-01, 1.92307692e+03, 8.07167649e+00, 1.02019996e+01,
       4.61723151e+00, 1.10521662e+01, 4.00224126e-01, 3.68053000e+01,
       1.19780562e+00])

In [9]:
y_train = pd.get_dummies(y_train['label'])
y_test = pd.get_dummies(y_test['label'])

In [10]:
X_submit = X_submit.values
X_train = X_train.values
X_test = X_test.values
y_train = np.squeeze(y_train.values)
y_test = np.squeeze(y_test.values)

X_train.shape

(80000, 310)

In [11]:
X_train = tf.convert_to_tensor(X_train, dtype='float64')
y_train = tf.convert_to_tensor(y_train, dtype='float64')
X_test  = tf.convert_to_tensor(X_test, dtype='float64')
y_test  = tf.convert_to_tensor(y_test, dtype='float64')

In [12]:
model = train(model, X_train, y_train, X_test, y_test)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Epoch: 0 	 train

In [10]:
model_name = '510_stack_nn'
model.save_weights('M_336/checkpoints/'+model_name+'/model')

In [13]:
X_test_pred   = np.squeeze(np.array(model(X_test)))
X_submit_pred = np.squeeze(np.array(model(X_submit)))

X_test_pred.shape

(20000, 13)

In [20]:
df_submit = pd.DataFrame.from_records(tf.nn.softmax(X_submit_pred).numpy())

cols = ['class'+str(i) for i in range(13)]

df_submit.columns = cols
df_submit.head()

,class0,class1,class2,class3,class4,class5,class6,class7,class8,class9,class10,class11,class12
0,0.021627,0.023334,0.065777,0.124921,0.044512,0.000417,0.048683,0.051649,0.077450,0.074571,0.275773,0.032790,0.158495
1,0.001412,0.063774,0.112211,0.138676,0.087546,0.000054,0.055819,0.417896,0.052942,0.036014,0.018985,0.009312,0.005359
2,0.008781,0.060230,0.217216,0.181244,0.162623,0.001515,0.042537,0.167549,0.043226,0.035115,0.053229,0.013571,0.013165
3,0.006011,0.134851,0.149127,0.139214,0.131760,0.000134,0.078139,0.203856,0.063280,0.042052,0.021489,0.023419,0.006669
4,0.085706,0.142549,0.124000,0.116376,0.091587,0.000119,0.084573,0.008571,0.060298,0.067366,0.098988,0.085695,0.034170


In [21]:
df_submit = pd.concat([
    y_submit.reset_index(),
    df_submit
], axis=1)

df_submit = df_submit.set_index('id')
df_submit

,class0,class1,class2,class3,class4,class5,class6,class7,class8,class9,class10,class11,class12
id,,,,,,,,,,,,,
151807,0.021627,0.023334,0.065777,0.124921,0.044512,0.000417,0.048683,0.051649,0.077450,0.074571,0.275773,0.032790,0.158495
118131,0.001412,0.063774,0.112211,0.138676,0.087546,0.000054,0.055819,0.417896,0.052942,0.036014,0.018985,0.009312,0.005359
110921,0.008781,0.060230,0.217216,0.181244,0.162623,0.001515,0.042537,0.167549,0.043226,0.035115,0.053229,0.013571,0.013165
105149,0.006011,0.134851,0.149127,0.139214,0.131760,0.000134,0.078139,0.203856,0.063280,0.042052,0.021489,0.023419,0.006669
143868,0.085706,0.142549,0.124000,0.116376,0.091587,0.000119,0.084573,0.008571,0.060298,0.067366,0.098988,0.085695,0.034170
...,...,...,...,...,...,...,...,...,...,...,...,...,...
146316,0.015957,0.115297,0.093594,0.102673,0.103073,0.000134,0.077883,0.078376,0.081684,0.133270,0.072542,0.088998,0.036518
121816,0.155807,0.013082,0.351903,0.129099,0.191808,0.000986,0.004530,0.000187,0.002742,0.004093,0.133633,0.007107,0.005022
106217,0.039621,0.199656,0.211499,0.118088,0.196356,0.000051,0.057648,0.009684,0.031820,0.034603,0.031910,0.063821,0.005244


In [22]:
# df_submit = pd.concat([
#     y_submit.reset_index(),
#     df_submit
# ], axis=1)

# df_submit = df_submit.set_index('id')
# df_submit

In [23]:
df_submit.to_csv('004_submit.csv')